In [1]:
# download tweets from stocktwits
import urllib.request, json
import os
import sys
import datetime
import time
from urllib.request import Request, urlopen

# fetch date and seq for csv file purpose
today=datetime.datetime.today().strftime('%Y%m%d')
seq=datetime.datetime.today().strftime('%H%M%S')


In [2]:
# fetch date and seq for csv file purpose
today=datetime.datetime.today().strftime('%Y%m%d')
seq=datetime.datetime.today().strftime('%H%M%S')

In [3]:
#os.getcwd()
data_dir='C:\\Users\\LENOVO\\Desktop\\Big data PI\\Stocktweets'
file=data_dir+"\\StockTwits."+today+"."+seq+".csv"

In [4]:
writer = open(file, 'w')
writer.write('ID,Symbol,CreateTime,Body,Sentiment\n')

36

In [7]:
!pip install nbconvert

In [5]:
#stocks=["AAPL", "TSLA", "FB"]
stocks=["FB"]

rows=0
batches=50

In [6]:
try:
    for stock in stocks:
        url=Request("https://api.stocktwits.com/api/2/streams/symbol/"+stock+".json", headers={'User-Agent' :'Mozilla/5.0'})
        print('------------------------------------------------------------------------------')
        print(url)
        with urllib.request.urlopen(url) as urlobj:
            json_data = json.loads(urlobj.read().decode())
            symbol=json_data['symbol']
            cursor=json_data['cursor']
            more=cursor['more']
            since=cursor['since']
            maxid=cursor['max']
            iter=0
            while more and iter < batches:
                #time.sleep(1) #sec
                print('iteration:{0} {1},{2},{3},{4}'.format(iter, stock, more, since, maxid))
                url=Request("https://api.stocktwits.com/api/2/streams/symbol/"+stock+".json?max="+str(maxid) , headers={'User-Agent' :'Mozilla/5.0'})
                urlobj2=urllib.request.urlopen(url)
                json_data = json.loads(urlobj2.read().decode())
                #symbol=json_data['symbol']
                cursor=json_data['cursor']
                more=cursor['more']
                since=cursor['since']
                maxid=cursor['max']
                iter=iter+1
                for message in json_data['messages']:
                    # replace ',' with '#' as output is csv file
                    # replace unprintable chars with unicode chars
                    
                    adj_body=message['body'].replace(",","#").encode(sys.stdout.encoding, errors='replace')
                    sentiment="None"
                    if message['entities']['sentiment']:
                        sentiment=message['entities']['sentiment']['basic']
                    if rows % 30 == 0:
                        print('    {0},{1},{2},{3},{4}'.format(message['id'], stock, message['created_at'], adj_body, sentiment))
                    # build the csv row
                    row=str(message['id'])+","+stock+","+message['created_at']+","+str(adj_body)+","+sentiment+"\n"
                    writer.write(row)
                    rows=rows+1
finally:
    writer.close()
            
print("Written {0} rows to file: {1}".format(rows, file)) 


------------------------------------------------------------------------------
iteration:0 FB,True,254650775,254636968
    254636703,FB,2020-11-05T17:56:14Z,b'New monthly highs \n$GE $TLRY $SOLO $FB $JMIA  \n \n+72 more results \nstockbeep.com/mhi-nov5p',None
iteration:1 FB,True,254636703,254627080
    254627065,FB,2020-11-05T17:21:15Z,b'$FB 290 in half an hour.',None
iteration:2 FB,True,254627065,254618268
    254617870,FB,2020-11-05T16:47:47Z,b'$FB gap to fill @ $298.60\xe2\xac\x86\xef\xb8\x8f\xf0\x9f\x90\x82',Bullish
iteration:3 FB,True,254617870,254612429
    254612424,FB,2020-11-05T16:28:20Z,b'$FB   Vol pop BBRG mentions that India permits WhatsApp payments service to go live (sm)',Bullish
iteration:4 FB,True,254612424,254603463
    254602917,FB,2020-11-05T15:56:55Z,b'$FB Shorties don\xe2\x80\x99t want to admit they started their positions at $275 \xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3',Bullish
iteration:5 FB,True,254602917,254593989
    254593212,FB,2020-11-05T15:27:34Z,b'$NVDA blood r